In [32]:
class LiftFinder():
    b = 0
    PI = 3.1415926536
    v_infinity = 0
    rho = 0
    c(x) = 0
    alphaL0(x) = 0
    alpha(x) = 0
    N = 0
    x_axis_reference = []
    lift_vector = []
    
    def define_vars(self, wing_info):
        self.b = wing_info[0]
        self.v_infinity = wing_info[1]
        self.rho = wing_info[2]
        self.c = SR(wing_info[3].replace("pi", str(self.PI)))
        self.alphaL0 = SR(wing_info[4].replace("pi", str(self.PI)))
        self.alpha = SR(wing_info[5].replace("pi", str(self.PI)))
        self.N = int(wing_info[6])


    def five_fifty_one(self):
        x = var("x")
        N = self.N
        alpha = self.alpha
        alphaL0 = self.alphaL0
        pi = self.PI
        b = self.b
        c = self.c
        print(alpha, c)
        rhs_vector = [0 for i in range(N)]
        eqParts = [[0 for i in range(N)] for i in range(N)]
        #The last column is the RHS, alpha(theta) - alphaL0(theta)
        for row in range(0, N):
            theta = pi * (row + 1)/(N+1) # This gets N non-zero thetas evenly distributed, using pi * row / N would have zero as the first value
            rhs_vector[row] = [alpha(x=theta) - alphaL0(x=theta)] # assign last column
            
            for col in range(0, N):
                n = col + 1
                eqParts[row][col] = (2 * b * sin(n * theta)) / (pi * c(x=theta)) + (n * sin(n * theta)) / sin(theta)

        an_vector = matrix(eqParts).solve_right(matrix(rhs_vector))
        return an_vector


    def gamma(N, alpha):
        pi = self.PI

        an_vector = five_fifty_one(self, N, alpha)
        gamma_per_theta = [0 for i in range(0, N)]
        for theta_lcv in range(0, N):
            sum = 0
            theta = pi * (theta_lcv + 1) / (N + 1)
            for i in range(0, N): #i is the loop control variable, n is what we're using in the actual sum
                n = i + 1
                sum = sum + an_vector[i] * sin(n * theta)
            gamma_per_theta[theta_lcv] = sum * 2 * b * v_infinity
        return gamma_per_theta


    def lift_finder(self, N, alpha):
        gamma_per_theta = gamma(N, alpha)
        lift_vector = []
        for value in gamma_per_theta:
            lift_vector.append(rho * v_infinity * abs(value))
        return (lift_vector)


    def get_total_lift(self, full_lift_vector, x_axis_reference):
        delta_x = x_axis_reference[1]
        total = 0
        for index in range(N + 1):
            avg = (full_lift_vector[index] + full_lift_vector[index + 1]) / 2
            total = total + delta_x * avg
            
    def generate_vectors(self):
        lift_vector = lift_vector = full_lift_vector(lift_finder(N, alpha))
        x_axis_reference = generate_x_axis(N)

    
    def get_plot(self):
       list_plot(list(zip(x_axis_reference, lift_vector)), axes_labels = ["Displacement", "Lift"] )

    def get_x_axis_reference(self):
        return x_axis_reference
    def get_lift_vector(self):
        return lift_vector
    
    # Some helper methods
    def generate_x_axis(self, N):
        pi = self.PI

        x_axis_reference = [- b / 2]
        for n in range(0, N):
            theta = pi * (n + 1)/(N+1)
            x_axis_reference.append(-b/2*(cos(theta)))

        x_axis_reference.append(b/2)
        return x_axis_reference
    def full_lift_vector(self, lift_vector):
        lift_per_y = [0]

        for value in lift_vector:
            lift_per_y.append(value)
        lift_per_y.append(0)
        
        return lift_per_y


In [33]:
plottables = []

wing_info = [20, 50, 1.225, "5", "0", "5 * pi/180", 100]
vec = LiftFinder()

vec.define_vars(wing_info)

print(vec.five_fifty_one())

# x_ax = vec.get_x_axis_reference
# y_ax = vec.get_lift_vector
#list_plot(list(zip(x_ax, y_ax)), axes_labels = ["Displacement", "Lift"] )

0.0872664626000000 5
[   0.0279397941964136]
[0.0000370021832003981]
[  0.00262675813918135]
[0.0000685770724054351]
[ 0.000510375972216674]
[0.0000193601142833617]
[0.0000941993640293944]
[ -3.95788193132982e-6]
[0.0000433020310292637]
[ -1.55402155988864e-6]
[0.0000275902576057776]
[ -5.42601815805961e-7]
[0.0000120722883269250]
[ -6.64706959843235e-6]
[  8.32050447372372e-6]
[ -5.30581289651326e-6]
[  7.78925601876548e-6]
[ -4.67947273972478e-6]
[  5.62619356416495e-6]
[ -2.18068034572149e-6]
[  1.35528966523291e-6]
[  1.88641186049854e-6]
[ -1.78501612926078e-6]
[  1.93544848374577e-6]
[  4.19381652734164e-7]
[ -8.86055307771278e-7]
[  1.22614608897108e-6]
[  4.42995577711994e-7]
[ -1.11789697496391e-6]
[  1.63155227827684e-6]
[ -7.64931871552202e-7]
[  5.95864127849028e-7]
[ -5.26308374513531e-7]
[  1.24733938458393e-6]
[ -1.43754434296775e-6]
[  1.75070859026823e-6]
[ -1.66990643566114e-6]
[  1.78130056858351e-6]
[ -1.42295979996314e-6]
[  1.16724547585010e-6]
[ -8.51663464037104